In [31]:
%pip install langchain_community langchainhub langchain 
%pip install langchain google-generativeai
%pip install faiss-cpu lancedb tiktoken
%pip install playwright

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [playwright]3 [playwright]
Note: you may need to restart the kernel to use updated packages.


In [24]:
#from langchain_community.document_loaders import WebBaseLoader

can't use webbase loader because it Coursera Reviews is in JavaScript and not just plain HTML.

Playwright is like a robot that can open a web browser and do stuff just like a human—click buttons, scroll pages, fill forms, and read content, even when that content is loaded with JavaScript.

Playwright is an open-source automation library developed by Microsoft that allows you to control modern web browsers (like Chromium, Firefox, and WebKit) using JavaScript

In [ ]:
from playwright.async_api import async_playwright

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

    #there are many review pages, so we need to loop through them
        # Start from the first page
        page_num = 1
        all_reviews = []

        while True:
            url = f"https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page={page_num}"
            print(f"Loading page {page_num}: {url}")
            await page.goto(url)

            try:
                # Wait for reviews to load or timeout if not found
                await page.wait_for_selector(".review-text", timeout=5000)
                reviews = await page.query_selector_all(".review-text")

            
                if not reviews:
                    print("No more reviews found. Stopping.")
                    break

                for review in reviews:
                    review_text = await review.inner_text()
                    all_reviews.append(review_text)

                page_num += 1

            except:
                #after the end page there won't be any review after page 37 for ex, and thus we will end the page then. 
                print("No reviews found or error occurred. Ending loop.")
                break

        # Save all reviews to a file so that we can split/chunk them later 
        with open('all_reviews.txt', 'w', encoding='utf-8') as file:
            for review in all_reviews:
                file.write(review.strip() + "\n\n")

        await browser.close()


await run()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading page 1: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=1
Loading page 2: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=2
Loading page 3: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=3
Loading page 4: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=4
Loading page 5: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=5
Loading page 6: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=6
Loading page 7: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=7
Loading page 8: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=8
Loading page 9: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=9
Loading page 10: https://www.coursera.org/learn/classification-vector-spaces-in-nlp/reviews?page=10
Loading page 11: h

In [50]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
with open('reviews.txt', 'r', encoding='utf-8') as file:
    reviews_text = file.read()

#now the splitting 
split=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks=split.split_text(reviews_text)

Testing the chunks, remove the # for test:-

In [51]:
#for i, chunk in enumerate(chunks):             
#    print(f"Chunk {i + 1}:\n{chunk}\n")

In [52]:
#Embedding the chunks
from langchain.embeddings import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [53]:
#now use lanceDB to store the embeddings
from langchain.vectorstores import LanceDB
vectorDB=LanceDB.from_texts(chunks, embedding)

In [ ]:
#setting up Gemini LLM 
import os
os.environ["GOOGLE_API_KEY"] = "input_your_api_key_here"
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
)

In [55]:
#now the qna pipeline
from langchain.chains import RetrievalQA
qna=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorDB.as_retriever()
)

In [ ]:
#example query, we can even modify it.
query="What is disadavantage of this course?"
result=qna.run(query)
print(result)

Based on the provided reviews, here are some disadvantages of the course:

*   **Lack of depth:** Some reviewers felt the course didn't go deep enough into the topics.
*   **Easy assignments:** The assignments were considered too easy, allowing students to pass without fully understanding the concepts.
*   **Unhelpful assignment:** The assignments were not helpful in furthering understanding.
*   **Uninspiring lectures:** The video lectures were described as dry, with the instructor sounding like they were just reading a script.
*   **Poor topic selection:** The course didn't dive deep enough into the right topics.
